In [235]:
#Importing necessary python libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [236]:
# connect to SQlite NHANES DB and read in necessary data.
con = sqlite3.connect("../data/NHANES SQLite DBs/2013-2014 NHANES Survey.db")
query = """

SELECT

SEQN, RIDAGEYR, RIAGENDR, RIDRETH3, BPXDI1, BPXDI2, BPXDI3, 
BPXDI4, BPXSY1, BPXSY2, BPXSY3, BPXSY4, BMXWT, BPXCHR, 
BPXPLS, BPXPULS, LBDHDD, LBDLDL, LBXTC, LBXTR, LBXSGL, 
MCQ300a, PAD680, RXQ520, SMQ040,SMQ020, DIQ010, MCQ160e, MCQ160c

FROM demographic
JOIN examination
    USING(SEQN)
JOIN labs
    USING(SEQN)
JOIN questionnaire
    USING(SEQN)
;
"""
heart_df = df = pd.read_sql(query, con)
heart_df.head()

,SEQN,RIDAGEYR,RIAGENDR,RIDRETH3,BPXDI1,BPXDI2,BPXDI3,BPXDI4,BPXSY1,BPXSY2,...,LBXTR,LBXSGL,MCQ300A,PAD680,RXQ520,SMQ040,SMQ020,DIQ010,MCQ160E,MCQ160C
0,73557,69,1,4,72,76,74,None,122,114,...,None,554,2,600,None,3,1,1,2,2
1,73558,54,1,3,62,80,42,None,156,160,...,None,219,2,540,None,2,1,1,2,2
2,73559,72,1,3,90,76,80,None,140,140,...,51,183,1,300,2,3,1,1,2,2
3,73560,9,1,3,38,34,38,None,108,102,...,None,None,None,None,None,None,None,2,None,None
4,73561,73,2,3,86,88,86,None,136,134,...,75,104,1,480,None,None,2,2,2,2


In [237]:
#Connect to SQLite NHANES Variable Names DB and read in variable name discriptions
con2 = sqlite3.connect("../data/NHANES SQLite DBs/2013-2014 NHANES Survey Varible Lists.db")

query2 = """
SELECT VariableName, VariableDescription, DataFileDescription, Component
FROM demographics_variable_list
UNION ALL
SELECT VariableName, VariableDescription, DataFileDescription, Component 
FROM examination_variable_list
UNION ALL
SELECT VariableName, VariableDescription, DataFileDescription, Component 
FROM laboratory_variable_list
UNION ALL
SELECT VariableName, VariableDescription, DataFileDescription, Component
FROM questionnaire_variable_list
;
"""
var_df = pd.read_sql_query(query2, con2)
var_df.head()

,VariableName,VariableDescription,DataFileDescription,Component
0,AIALANGA,Language of the MEC ACASI Interview Instrument,Demographic Variables and Sample Weights,Demographics
1,DMDBORN4,In what country {were you/was SP} born?,Demographic Variables and Sample Weights,Demographics
2,DMDCITZN,{Are you/Is SP} a citizen of the United States...,Demographic Variables and Sample Weights,Demographics
3,DMDEDUC2,What is the highest grade or level of school {...,Demographic Variables and Sample Weights,Demographics
4,DMDEDUC3,What is the highest grade or level of school {...,Demographic Variables and Sample Weights,Demographics


In [238]:
#Close SQLite DB connections
con.close()
con2.close()

In [239]:
#Function that easily identifies the description of a specific variable in one of the NHANES DataFrames above.
def var_desc(var_name):
    """
    This funtions takes in a variable name 
    from the NHANES survey data and returns 
    the variable's description. 
    The variable must be entered as a string.
    
    """
    df = desc = var_df[var_df['VariableName'] == var_name]
    comp = df.Component.iloc[0]
    desc = df.VariableDescription.iloc[0]
    print(f'Table: {comp}')
    print(f'Variable Description: "{desc}"')

In [240]:
var_desc('PAD615')

Table: Questionnaire
Variable Description: "How much time {do you/does SP} spend doing vigorous-intensity activities at work on a typical day?"


In [241]:
# rename columns
col_rename ={
'RIDAGEYR': 'age', 'RIAGENDR': 'gender', 'RIDRETH3': 'race/ethnicity', 'BPXDI1': 'dia bp 1', 'BPXDI2': 'dia bp 2', 
'BPXDI3': 'dia bp 3', 'BPXDI4': 'dia bp 4', 'BPXSY1': 'sys bp 1', 'BPXSY2': 'sys bp 2', 'BPXSY3': 'sys bp 3', 
'BPXSY4': 'sys bp 4', 'BMXWT': 'weight (kg)', 'BPXCHR': 'hr', 'BPXPLS': 'pulse', 'BPXPULS': 'pulse irregular', 'LBDHDD': 'hdl chol (mg/dL)', 
'LBDLDL': 'ldl chol (mg/dL)', 'LBXTC': 'tot chol (mg/dL)', 'LBXTR': 'trigl (mg/dL)', 'LBXSGL': 'gluc (mg/dL)', 
'MCQ300A': 'fam hist ha or ang < age 50)', 'PAD680': 'time sedetary (min)', 'RXQ520': 'prev aspirin use', 'SMQ040': 'smoker', 
'SMQ020': 'smoked >= 100','DIQ010': 'diabetes', 'MCQ160E': 'heart attack', 'MCQ160C': 'coronary heart disease'
}

heart_df.rename(columns=col_rename, inplace=True)
heart_df.head()

,SEQN,age,gender,race/ethnicity,dia bp 1,dia bp 2,dia bp 3,dia bp 4,sys bp 1,sys bp 2,...,trigl (mg/dL),gluc (mg/dL),fam hist ha or ang < age 50),time sedetary (min),prev aspirin use,smoker,smoked >= 100,diabetes,heart attack,coronary heart disease
0,73557,69,1,4,72,76,74,None,122,114,...,None,554,2,600,None,3,1,1,2,2
1,73558,54,1,3,62,80,42,None,156,160,...,None,219,2,540,None,2,1,1,2,2
2,73559,72,1,3,90,76,80,None,140,140,...,51,183,1,300,2,3,1,1,2,2
3,73560,9,1,3,38,34,38,None,108,102,...,None,None,None,None,None,None,None,2,None,None
4,73561,73,2,3,86,88,86,None,136,134,...,75,104,1,480,None,None,2,2,2,2


In [217]:
#Replace none with NaN for easier manipulation
heart_df.fillna(value=np.nan, inplace=True)
heart_df

,SEQN,age,gender,race/ethnicity,dia bp 1,dia bp 2,dia bp 3,dia bp 4,sys bp 1,sys bp 2,...,trigl (mg/dL),gluc (mg/dL),fam hist ha or ang < age 50),time sedetary (min),prev aspirin use,smoker,smoked >= 100,diabetes,heart attack,coronary heart disease
0,73557,69,1,4,72,76,74,NaN,122,114,...,NaN,554,2,600,NaN,3,1,1,2,2
1,73558,54,1,3,62,80,42,NaN,156,160,...,NaN,219,2,540,NaN,2,1,1,2,2
2,73559,72,1,3,90,76,80,NaN,140,140,...,51,183,1,300,2,3,1,1,2,2
3,73560,9,1,3,38,34,38,NaN,108,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
4,73561,73,2,3,86,88,86,NaN,136,134,...,75,104,1,480,NaN,NaN,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,83727,26,1,2,68,74,76,NaN,110,116,...,81,101,2,600,NaN,NaN,2,2,2,2
9809,83728,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
9810,83729,42,2,4,82,82,80,NaN,136,130,...,NaN,112,2,720,2,NaN,2,2,2,2
9811,83730,7,1,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN


In [218]:
heart_df = heart_df.astype(float)

In [219]:
avg_sys_bp = heart_df[['sys bp 1', 'sys bp 2', 'sys bp 3', 'sys bp 4']].mean(axis=1).round(0)
avg_dia_bp = heart_df[['dia bp 1', 'dia bp 2', 'dia bp 3', 'dia bp 4']].mean(axis=1).round(0)
pulse = heart_df[['hr', 'pulse']].mean(axis=1).round(0)

In [220]:
heart_df.drop(columns=['dia bp 1', 'dia bp 2', 'dia bp 3', 'dia bp 4', 'sys bp 1', 'sys bp 2', 'sys bp 3', 'sys bp 4', 'hr', 'pulse'], inplace=True)
heart_df.insert(5, 'sys bp', avg_sys_bp)
heart_df.insert(6, 'dia bp', avg_dia_bp)
heart_df.insert(7, 'pulse', pulse)
heart_df.head(15)

,SEQN,age,gender,race/ethnicity,weight (kg),sys bp,dia bp,pulse,pulse irregular,hdl chol (mg/dL),...,trigl (mg/dL),gluc (mg/dL),fam hist ha or ang < age 50),time sedetary (min),prev aspirin use,smoker,smoked >= 100,diabetes,heart attack,coronary heart disease
0,73557.0,69.0,1.0,4.0,78.3,113.0,74.0,86.0,1.0,65.0,...,NaN,554.0,2.0,600.0,NaN,3.0,1.0,1.0,2.0,2.0
1,73558.0,54.0,1.0,3.0,89.5,157.0,61.0,74.0,1.0,50.0,...,NaN,219.0,2.0,540.0,NaN,2.0,1.0,1.0,2.0,2.0
2,73559.0,72.0,1.0,3.0,88.9,142.0,82.0,68.0,1.0,60.0,...,51.0,183.0,1.0,300.0,2.0,3.0,1.0,1.0,2.0,2.0
3,73560.0,9.0,1.0,3.0,32.2,105.0,37.0,64.0,1.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,73561.0,73.0,2.0,3.0,52.0,137.0,87.0,92.0,1.0,85.0,...,75.0,104.0,1.0,480.0,NaN,NaN,2.0,2.0,2.0,2.0
5,73562.0,56.0,1.0,1.0,105.0,157.0,82.0,60.0,1.0,38.0,...,NaN,104.0,2.0,360.0,NaN,3.0,1.0,2.0,1.0,1.0
6,73563.0,0.0,1.0,3.0,7.4,NaN,NaN,152.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,73564.0,61.0,2.0,3.0,93.4,123.0,81.0,82.0,1.0,58.0,...,64.0,107.0,2.0,60.0,NaN,NaN,2.0,2.0,2.0,2.0
8,73566.0,56.0,2.0,3.0,61.8,122.0,73.0,86.0,1.0,59.0,...,NaN,108.0,2.0,30.0,2.0,1.0,1.0,2.0,2.0,2.0
9,73567.0,65.0,1.0,3.0,65.3,141.0,77.0,70.0,1.0,79.0,...,NaN,104.0,1.0,480.0,2.0,1.0,1.0,2.0,2.0,2.0


In [203]:
heart_df['smoker'].where((heart_df['smoker'].notna()) & (heart_df['smoked >= 100'] != 2.0), heart_df['smoked >= 100'], inplace=True)
heart_df['smoker'].isna().sum()

3889

Series([], Name: diabetes, dtype: int64)